## Chapter 1 Table of Contents

* [1.1 Data Cleaning - Whisky](#1_1)
* [1.2 Data Cleaning - Food](#1_2)

In [1]:
import numpy as np
import pandas as pd

import string
import regex as re
import nltk

stopwords = nltk.corpus.stopwords.words('english')
ps = nltk.PorterStemmer()
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer

import matplotlib.pyplot as plt
import seaborn as sns

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

## 1.1 Data Cleaning - Whisky<a class="anchor" id="1_1"></a>

In [2]:
# load data
df = pd.read_csv('data/whiskyanalysis3.csv')

In [3]:
df.head()

,whisky,normalised_rating,stdev,num_of_reveiws,price_category,class,super_cluster,cluster,body,sweetness,base_flavour,winey,fruity,floral,honey,malty,nutty,spicy,smoky,medicinal,some_fruity,some_honey,some_malty,some_nutty,some_spicy,some_smoky,some_medicinal,country,type
0,"Macallan 10yo Full Proof 57% 1980 (OB, Giovine...",9.57,0.24,3,6,SingleMalt-like,ABC,A,full,sweet,sherry,0.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,Scotland,Malt
1,Ledaig 42yo Dusgadh,9.48,0.23,3,6,SingleMalt-like,ABC,C,full,sweet,sherry,0.0,1.0,1.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,Scotland,Malt
2,"Laphroaig 27yo 57.4% 1980-2007 (OB, 5 Oloroso ...",9.42,0.23,4,6,SingleMalt-like,ABC,C,full,sweet,sherry,0.0,1.0,1.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,Scotland,Malt
3,Glenfarclas 40yo,9.29,0.26,17,6,SingleMalt-like,ABC,A,full,sweet,sherry,0.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,Scotland,Malt
4,Glengoyne 25yo,9.24,0.22,21,6,SingleMalt-like,ABC,A,full,sweet,sherry,0.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,Scotland,Malt


In [4]:
df.shape

(1787, 29)

In [5]:
df.isnull().sum().sort_values(ascending=False)

super_cluster        593
honey                316
some_honey           316
spicy                316
nutty                316
malty                316
some_fruity          316
floral               316
fruity               316
winey                316
some_malty           316
medicinal            316
some_nutty           316
some_spicy           316
some_smoky           316
some_medicinal       316
smoky                316
cluster              315
country                0
whisky                 0
normalised_rating      0
base_flavour           0
sweetness              0
body                   0
class                  0
price_category         0
num_of_reveiws         0
stdev                  0
type                   0
dtype: int64

In [6]:
df = df.drop(['super_cluster'], axis=1)

In [7]:
# label unspecified values as 0
df = df.fillna(0)

In [8]:
df.isnull().sum().sort_values(ascending=False)

whisky               0
normalised_rating    0
country              0
some_medicinal       0
some_smoky           0
some_spicy           0
some_nutty           0
some_malty           0
some_honey           0
some_fruity          0
medicinal            0
smoky                0
spicy                0
nutty                0
malty                0
honey                0
floral               0
fruity               0
winey                0
base_flavour         0
sweetness            0
body                 0
cluster              0
class                0
price_category       0
num_of_reveiws       0
stdev                0
type                 0
dtype: int64

In [9]:
# re-label cluster A, B, C, E, F, G, H, I, J, R0, R1, R2, R3, R4 as 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14
df['cluster'] = df['cluster'].replace(['A','B','C','E','F','G','H','I','J','R0','R1','R2','R3','R4' ],[1,2,3,4,5,6,7,8,9,10,11,12,13,14])

In [10]:
# dummify the columns 'class', 'body', 'sweetness', 'base_flavour', 'country', 'type'
df = pd.get_dummies(df, columns=['class', 'body', 'sweetness', 'base_flavour', 'country', 'type'])

In [11]:
# change all column headers to lowercase and replace all spaces with '_'
df.columns = df.columns.str.lower().str.replace(' ', '_')

In [12]:
df.head()

,whisky,normalised_rating,stdev,num_of_reveiws,price_category,cluster,winey,fruity,floral,honey,malty,nutty,spicy,smoky,medicinal,some_fruity,some_honey,some_malty,some_nutty,some_spicy,some_smoky,some_medicinal,class_bourbon-like,class_rye-like,class_scotch-like,class_singlemalt-like,body_full,body_light,body_medium,body_very_light,sweetness_dry,sweetness_medium-sweet,sweetness_sweet,base_flavour_aperitif-style,base_flavour_malty,base_flavour_quite_smoky,base_flavour_sherry,base_flavour_spicy,base_flavour_very_smoky,country_belgium,country_canada,country_england,country_finland,country_france,country_india,country_ireland,country_japan,country_netherlands,country_scotland,country_south_africa,country_sweden,country_switzerland,country_taiwan,country_tasmania,country_usa,country_wales,type_barley,type_blend,type_bourbon,type_grain,type_malt,type_rye,type_wheat,type_whiskey
0,"Macallan 10yo Full Proof 57% 1980 (OB, Giovine...",9.57,0.24,3,6,1,0.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0,1,1,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0
1,Ledaig 42yo Dusgadh,9.48,0.23,3,6,3,0.0,1.0,1.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0,0,0,1,1,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0
2,"Laphroaig 27yo 57.4% 1980-2007 (OB, 5 Oloroso ...",9.42,0.23,4,6,3,0.0,1.0,1.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0,0,0,1,1,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0
3,Glenfarclas 40yo,9.29,0.26,17,6,1,0.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0,1,1,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0
4,Glengoyne 25yo,9.24,0.22,21,6,1,0.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0,1,1,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0


In [13]:
df.isnull().sum().sort_values(ascending=False)

whisky                         0
normalised_rating              0
base_flavour_malty             0
base_flavour_quite_smoky       0
base_flavour_sherry            0
base_flavour_spicy             0
base_flavour_very_smoky        0
country_belgium                0
country_canada                 0
country_england                0
country_finland                0
country_france                 0
country_india                  0
country_ireland                0
country_japan                  0
country_netherlands            0
country_scotland               0
country_south_africa           0
country_sweden                 0
country_switzerland            0
country_taiwan                 0
country_tasmania               0
country_usa                    0
country_wales                  0
type_barley                    0
type_blend                     0
type_bourbon                   0
type_grain                     0
type_malt                      0
type_rye                       0
type_wheat

In [14]:
df.to_csv('data/whiskyanalysis_cleaned.csv', index=False)

## 1.2 Data Cleaning - Food<a class="anchor" id="1_2"></a>

In [15]:
# load data
df2 = pd.read_csv('data/whisky_food.csv')

In [16]:
# label unspecified values as 0
df2 = df2.fillna(0)
df2.head()

,category,food,drink_pairing,cluster_1,cluster_2,cluster_3,cluster_4,cluster_5,cluster_6,cluster_7,cluster_8,cluster_9,cluster_10,cluster_11,cluster_12,cluster_13,cluster_14
0,cheese,hard blue cheese,spicy malty whiskies,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,cheese,salty cheddar cheese,aged scotch with honey and vanilla notes,1.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,cheese,creamy cow cheese,light smooth fragrant whiskies,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,cheese,classic goat cheese,medium bodied whiskies matured in oak and sher...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,cheese,smoked cheese,peaty whiskies,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0


In [17]:
df2.to_csv('data/whisky_food_cleaned.csv', index=False)